### 1. 주요 매직 메서드
* 매직 메서드는 Python 클래스에서 특별한 이름을 가진 메서드
* 특정 연산자나 내장 함수의 동작을 커스터마이징할 때 사용
1. \_\_init__(self): 객체 생성 시 호출되는 초기화 메서드
2. \_\_str__(self): print() 함수로 객체를 출력할 때 호출되는 메서드 (객체의 문자열 표현을 정의)
3. \_\_add__(self, other): 두 객체를 더할 때 호출됩니다. 주로 연산자를 오버로딩할 때 사용
4. \_\_len__(self): len() 함수가 호출될 때 사용

In [ ]:
class Person:
    def __init__(self,name,age):    # 객체 생성 시 호출되는 초기화 메서드
        self.name = name
        self.age = age
    def __str__(self):      # 객체의 문자열 표현 정의
        return f"Name : {self.name}, Age : {self.age}"
    def __add__(self, other):
        return self.age + other.age
    def __len__(self):
        return len(self.name)

person = Person("mksd",23)
person2 = Person("AA",26)
print(person.name)      # 객체의 이름
print(person)           # 객체의 str 값
print(person+person2)   # person.age + person2.age
print(len(person))      # 객체 이름의 길이

### 2. 특정 용도 데코레이터 만들기
* 데코레이터 : 어떤 함수가 있을 때 해당 함수를 직접 수정하지 않고 함수에 기능을 추가하고자 할 때 데코레이터를 사용
* 함수의 앞 뒤로 다른 코드를 사용할 수 있고, 함수의 리턴값도 조절 가능
* 함수의 인자를 검사하는 함수를 만들어 데코레이터 여러개 적용하여 활용 가능
1. 함수의 실행 시간을 측정해서 출력해주는 데코레이터
    * time 모듈을 사용하여 함수 실행 시간을 측정하는 데코레이터를 작성 가능
2. 함수를 실행하기 전에 사용자에게 진행 여부를 확인하는 데코레이터
    * 사용자에게 확인을 요청하는 데코레이터 생성 가능

In [ ]:
# 데코레이터의 사용 (함수의 리턴값을 출력할 수 있음.)
def decorator(func):
    def wrapper(*args, **kwargs):  # *args : 가변인자(튜플), **kwargs : 키워드 가변인자(딕셔너리) -> 함수마다 인자가 다를 수 있기 때문.
        print("함수 실행 전")
        func()
        print("함수 실행 후")
        # return func()   # func의 리턴 값이 있는 경우 반환하기 위해 사용
    return wrapper

@decorator
def num():
    print("안녕하세요!")
    return 3                # return func()를 주석처리해서 출력 값 None
print(num())

In [ ]:
# 데코레이터의 사용 (함수의 인자가 정수인지, 양수인지 판단하는 데코레이터)
def isInteger(func):
    def wrapper(arg1, arg2):
        if not (isinstance(arg1, int) and isinstance(arg2, int)):
            raise ValueError("인자가 정수가 아님!")
        return func(arg1,arg2)
    return wrapper

def isPositive_num(func):
    def wrapper(* arg):
        for i in arg:
            if i <= 0:
                raise ValueError("인자가 양수가 아님!")
        return func(* arg)
    return wrapper

@isInteger      # 데코레이터 선언 -> 함수의 인자가 정수인지 검사
@isPositive_num # 함수의 인자가 양수인지 검사
def add_num(num1,num2):
    return num1 + num2
def add_num2(num1,num2):
    return num1+num2

# 데코레이터 사용하는 경우
print(add_num(1,2))
# 데코레이터 사용 X인 경우
add_func = isInteger(add_num2)
print(add_func(1,2))

In [ ]:
# 1. 함수의 실행 시간을 측정하는 데코레이터 
import time

def timer(func):                            # 함수 실행 시간 측정
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)      # func()의 리턴 값 저장 -> 여기서 func 한 번 호출
        print(type(result))         
        end_time = time.time()
        print(f"함수 : '{func.__name__}' 실행에 {end_time - start_time:.3f}초가 걸림.")
        return result                       # func()의 리턴 값 반환 -> 만약 func를 하면 함수의 객체가 반환되고 func()를 하면 함수가 2번 실행됨.
    return wrapper                          # func에 반환 값이 없다면 return result를 주석처리해도 실행 결과가 같음

@timer
def sleep_function():
    time.sleep(2)       # 2초 동안 동작 정지
    print("함수 실행 종료")

sleep_function()

In [ ]:
# 2. 함수를 실행하기 전에 사용자에게 진행 여부를 확인하는 데코레이터

def check(func):
    def wrapper(*args,**kwargs):
        data = input("함수를 실행할려면 y를 입력하세요!")
        if data == 'y' or data == 'Y':
            return func(*args, **kwargs)    # 여기서 함수 실행
        else:
            print("함수 실행이 취소되었습니다.")
    return wrapper
@check
def Hello(name):
    print(f"Hello, {name}!!")
Hello("mksd")

### 3. 게터와 세터를 사용하는 이유에 대해 예시
* 게터(getter)와 세터(setter)는 클래스 객체의 속성에 접근하거나 수정할 때 사용. 
* 이들은 객체에 직접적인 접근을 제한하면서 객체의 속성을 설정, 수정할 수 있게 함. 

##### 사용 이유 : 
1. 속성 값의 유효성 검증 : 값을 설정하기 전 유효성 검사
2. 읽기 전용 속성 : 값을 읽기만 할 수 있고 수정할 수 없도록 제한할 수 있음
3. 내부 데이터 캡슐화 : 클래스 외부에서 직접 접근하는 것보다 메서드를 통해 접근 -> 내부 구현을 변경할 때 더 유연하게 대처 가능

In [ ]:
class Person:
    def __init__(self, name, age):
        self._name = name   # 속성 은닉 (외부에서 접근 불가)
        self._age = age

    @property               # 데코레이터로 게터 선언 (은닉 속성의 값 획득 가능)
    def name(self):         # 이름은 게터 선언을 하지 않아 변경 불가.
        return self._name
    
    @property               # 데코레이터로 게터 선언 (은닉 속성의 값 획득 가능)
    def age(self):
        return self._age

    @age.setter             # 세터 선언 (값 변경 가능)
    def age(self, value):
        if not value > 0:   # 변경 값 미리 유효한지 검사
            raise ValueError("나이는 양수이어야만 합니다.")
        self._age = value

person = Person("mksd", 23)
print(f"변경 전 나이 : {person.age}")  # 23

person.age = 30  # 값 변경
print(f"변경 후 나이 : {person.age}")  # 30

try:
    person.age = -5  # 유효성 검사 실패
except ValueError as e:
    print(e)  # 나이는 양수이어야만 합니다.
else:
    try:
        person.name = "MMMKKK"              # 이름은 변경 불가 (읽기만 가능)
    except Exception as e:
        print("이름은 변경할 수 없습니다!")     


### 4. IS-A, HAS-A
1. is-a : 상속, "~은 ~의 한 종류이다. "를 의미
* 노트북은 컴퓨터의 종류다. -> computer 클래스와 Laptop 클래스는 is-a 관계
* is-a 관계를 표현할 때, 상속을 사용
2. has-a : 구성, "~이 ~을 가지거나 포함한다. "를 의미
* 컴퓨터는 cpu와 ram을 가지고 있음 -> computer has cpu -> has-a 관계

In [1]:
# is-a 관계
# 컴퓨터 클래스
class Computer:
    def __init__(self, cpu, ram):
        # 인스턴스 멤버
        self.CPU = cpu
        self.RAM = ram

    # 웹 서핑을 하는 메서드
    def browse(self):
        print('browse')


    # 일을 하는 메서드
    def work(self):
        print('work')


# 노트북 클래스
# (Computer) <- 컴퓨터 클래스를 상속한다
# 컴퓨터 클래스의 모든 멤버와 메서드를 가짐.
class Laptop(Computer):
    # 멤버 추가
    def __init__(self, cpu, ram, battery):
        # super는 현재 클래스의 슈퍼 클래스 즉, 기본 클래스를 의미
        super().__init__(cpu, ram)
        self.battery = battery

    # 메서드 추가
    def move(self, to):
        print('move to {}'.format(to))

# 테스트 코드
if __name__ == '__main__':
    lap = Laptop('intel', 16, 'powerful')
    lap.browse()
    lap.work()
    lap.move('office')

browse
work
move to office


In [ ]:
# has-a 관계 (합성)
class CPU:
    pass

class RAM:
    pass
class Computer:
    def __init__(self):
        # 생성자에서 CPU객체를 생성하여 멤버 cpu에 할당
        self.cpu = CPU()
        self.ram = RAM()


In [ ]:
# has-a 관계 (통합)

class Gun:
    def __init__(self, kind):
        self.kind = kind

    def bang(self):
        print('bang bang')


class Police:
    def __init__(self):
        # Police 인스턴스가 만들어질때 Gun객체를 갖고 있지 않다.
        self.gun = None

    # acquire_gun 메서드를 통해 Gun객체를 멤버로 가지게 됨.
    def acquire_gun(self, gun):
        self.gun = gun

	# release_gun 메서드를 통해 가지고 있던 총을 반납할수도 있음.
    def release_gun(self):
        gun = self.gun
        self.gun = None
        return Gun

    def shoot(self):
        if self.gun:
            self.gun.bang()
        else:
            print('Unable to shoot')


p1 = Police()
print('p1 shoots')
p1.shoot()
print('')

# p1은 아직 총을 소유하지 않음
revolver = Gun('Revolver')
# p1이 revolver를 획득
p1.acquire_gun(revolver)
# 이제 p1이 총을 소유하므로
# revolver는 None이 된다
revolver = None
print('p1 shoots again')
p1.shoot()
print('')

# p1이 총을 반납했으므로
# 더 이상 총을 소유하지 않는다
revolver = p1.release_gun()
print('p1 shoots again')
p1.shoot()

p1 shoots
Unable to shoot

p1 shoots again
bang bang

p1 shoots again
Unable to shoot


### 5. 스택, 큐
* 데코레이터 : 어떤 함수가 있을 때 해당 함수를 직접 수정하지 않고 함수에 기능을 추가하고자 할 때 데코레이터를 사용
* 함수의 앞 뒤로 다른 코드를 사용할 수 있고, 함수의 리턴값도 조절 가능